In [1]:
import pandas as pd 
from prophet import Prophet
import numpy as np 
import plotly.graph_objs as go

In [2]:
df = pd.read_csv("../data/root_ts.csv")
df = df.rename({"Unnamed: 0":"day_index"})
df.columns = ["day_index",'sales_quantity']

In [108]:
sample_submission = pd.read_csv('../data/sample_submission.csv')
sample_submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
df['day_index'].iloc[-1]

'd_1913'

In [3]:
calender = pd.read_csv('../data/calendar.csv')
calender.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [47]:
df_merged = df.merge(calender[['d','date']], right_on='d', left_on='day_index', how = 'left')
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1913 entries, 0 to 1912
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   day_index       1913 non-null   object
 1   sales_quantity  1913 non-null   int64 
 2   d               1913 non-null   object
 3   date            1913 non-null   object
dtypes: int64(1), object(3)
memory usage: 74.7+ KB


In [48]:
df_merged.head()

,day_index,sales_quantity,d,date
0,d_1,32631,d_1,2011-01-29
1,d_2,31749,d_2,2011-01-30
2,d_3,23783,d_3,2011-01-31
3,d_4,25412,d_4,2011-02-01
4,d_5,19146,d_5,2011-02-02


In [49]:
df_merged.tail()

,day_index,sales_quantity,d,date
1908,d_1909,35343,d_1909,2016-04-20
1909,d_1910,35033,d_1910,2016-04-21
1910,d_1911,40517,d_1911,2016-04-22
1911,d_1912,48962,d_1912,2016-04-23
1912,d_1913,49795,d_1913,2016-04-24


In [6]:
df_merged['date'] = pd.to_datetime(df_merged['date'], format='%Y%m%d', errors='ignore')
df_merged = df_merged[['date','sales_quantity']]
df_merged.columns = ['ds','y']
df_merged.head()

,ds,y
0,2011-01-29,32631
1,2011-01-30,31749
2,2011-01-31,23783
3,2011-02-01,25412
4,2011-02-02,19146


In [7]:
df_merged.tail()

,ds,y
1908,2016-04-20,35343
1909,2016-04-21,35033
1910,2016-04-22,40517
1911,2016-04-23,48962
1912,2016-04-24,49795


In [8]:
model = Prophet(daily_seasonality=True)
model.fit(df_merged)

/Users/ericliu/Library/Python/3.8/lib/python/site-packages/prophet/forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


Initial log joint probability = -27.3885
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        4390.2   0.000462918       88.5894      0.3819      0.3819      123   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     187       4393.31   7.18846e-05       123.548    4.57e-07       0.001      287  LS failed, Hessian reset 
     199       4393.44    0.00030333       97.6894           1           1      303   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     245       4393.83   0.000638502       324.915   8.284e-06       0.001      411  LS failed, Hessian reset 
     299       4394.17   7.02267e-05       51.0474           1           1      481   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     317        4394.3   0.000949357         209.5   1.318e-05       0.001      576  LS failed, Hessian rese

In [51]:
future = model.make_future_dataframe(periods=28)
future.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941 entries, 0 to 1940
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      1941 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 15.3 KB


In [52]:
future.head()

,ds
0,2011-01-29
1,2011-01-30
2,2011-01-31
3,2011-02-01
4,2011-02-02


In [53]:
future.tail(28)

,ds
1913,2016-04-25
1914,2016-04-26
1915,2016-04-27
1916,2016-04-28
1917,2016-04-29
1918,2016-04-30
1919,2016-05-01
1920,2016-05-02
1921,2016-05-03
1922,2016-05-04


In [54]:
future.loc[1913].values[0]

numpy.datetime64('2016-04-25T00:00:00.000000000')

In [55]:
calender[calender['d'] == 'd_1941']

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1940,2016-05-22,11617,Sunday,2,5,2016,d_1941,NaN,NaN,NaN,NaN,0,0,0


In [56]:
forecasting = model.predict(future)

/Users/ericliu/Library/Python/3.8/lib/python/site-packages/prophet/forecaster.py:896: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/ericliu/Library/Python/3.8/lib/python/site-packages/prophet/forecaster.py:896: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [57]:
# evaluation: 
sales_train_evaluaiton = pd.read_csv('../data/sales_train_evaluation.csv')
cols = sales_train_evaluaiton.columns[:]
sales_train_evluation_leaf = sales_train_evaluaiton.groupby('id').sum()
sales_train_evluation_leaf


,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_evaluation,3,0,0,1,4,2,0,2,0,0,...,2,3,1,0,0,0,1,0,0,0
FOODS_1_001_CA_2_evaluation,2,0,0,0,1,0,8,0,4,1,...,1,0,0,1,1,0,0,1,2,0
FOODS_1_001_CA_3_evaluation,1,2,1,1,1,2,0,1,1,1,...,1,2,2,0,0,1,0,3,2,2
FOODS_1_001_CA_4_evaluation,0,1,1,1,1,1,0,0,0,0,...,1,0,0,0,0,1,1,0,0,0
FOODS_1_001_TX_1_evaluation,0,1,1,0,0,0,0,0,2,1,...,1,1,1,1,5,0,2,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_516_TX_2_evaluation,0,1,0,0,0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,0,1
HOUSEHOLD_2_516_TX_3_evaluation,2,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,2
HOUSEHOLD_2_516_WI_1_evaluation,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [58]:
valid_root = sales_train_evluation_leaf[sales_train_evluation_leaf.columns[-28:]].sum(axis = 0)
valid_root = pd.DataFrame(valid_root,)
valid_root.columns = ['sales_quantity']
valid_root

,sales_quantity
d_1914,38793
d_1915,35487
d_1916,34445
d_1917,34732
d_1918,42896
d_1919,50429
d_1920,53032
d_1921,43181
d_1922,44314
d_1923,39601


In [59]:
calender[calender['d'] == 'd_1914']

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1913,2016-04-25,11613,Monday,3,4,2016,d_1914,NaN,NaN,NaN,NaN,0,0,0


In [60]:
y_hat_valid_df = forecasting.tail(28)
y_hat_lower = y_hat_valid_df['yhat_lower']
y_hat_upper= y_hat_valid_df['yhat_upper']
y_hat =  y_hat_valid_df['yhat']
x = y_hat_valid_df['ds']
y = valid_root['sales_quantity']

In [61]:
def plot_forecast(x, y_hat, y_hat_lower, y_hat_upper, y):

    x = x.apply(lambda x: x.strftime('%Y-%m-%d'))

    fig = go.Figure([
        go.Scatter(
            name='y_hat',
            x=x,
            y=y_hat,
            mode='lines',
            line=dict(color='rgb(31, 119, 180)'),
        ),
        go.Scatter(
            name='Upper Bound',
            x=x,
            y=y_hat_upper,
            mode='lines',
            marker=dict(color="#444"),
            line=dict(width=0),
            showlegend=False
        ),
        go.Scatter(
            name='Lower Bound',
            x=x,
            y=y_hat_lower,
            marker=dict(color="#444"),
            line=dict(width=0),
            mode='lines',
            fillcolor='rgba(68, 68, 68, 0.3)',
            fill='tonexty',
            showlegend=False
        ),
        go.Scatter(
            name='y',
            x=x,
            y=y,
            mode='lines',
            line=dict(color='rgb(25, 190, 70)'),
        ),
    ])

    fig.update_layout(
        yaxis_title='Forecasted Sales Quantity',
        title='Comparison',
        hovermode="x"
    )

    
    fig.show()

In [62]:
plot_forecast(x, y_hat, y_hat_lower, y_hat_upper,y)

In [64]:
sales_train_validation = pd.read_csv('../data/sales_train_validation.csv')
sales_train_validation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [82]:
sales_train_validation_sales = sales_train_validation.drop(columns = ['item_id','dept_id','cat_id','store_id','state_id'])
sales_train_validation_sales.head()

,id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [86]:
sales_train_validation_sales_np = sales_train_validation_sales.values
sales_train_validation_sales_np = sales_train_validation_sales_np[:,1:]
print(type(sales_train_validation_sales_np))
print(sales_train_validation_sales_np.shape)

<class 'numpy.ndarray'>
(30490, 1913)


In [93]:
sum_by_day = sales_train_validation_sales_np.sum(axis = 0)
percentage_by_day = sales_train_validation_sales_np/sum_by_day 
print(percentage_by_day.shape)

(30490, 1913)


In [99]:
historical_percentage = percentage_by_day.mean(axis = 1, keepdims=True )
print(historical_percentage.shape)

(30490, 1)


In [102]:
y_hat = y_hat.values.reshape(-1,1)
print(y_hat.shape)

(28, 1)


In [103]:
leaf_forecast = np.dot(historical_percentage, np.transpose(y_hat))
print(leaf_forecast.shape)

(30490, 28)


In [112]:
sales_train_validation_sales.columns

Index(['id', 'd_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9',
       ...
       'd_1904', 'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910',
       'd_1911', 'd_1912', 'd_1913'],
      dtype='object', length=1914)

In [124]:
result_float = pd.DataFrame(leaf_forecast, index=sales_train_validation_sales['id'], columns = sample_submission.columns[1:] )
result_float = result_float.astype(float)
result_float 

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_validation,0.330914,0.309658,0.306244,0.307492,0.340842,0.402224,0.398394,0.328874,0.307748,0.304468,...,0.339529,0.401431,0.398173,0.329277,0.308828,0.306280,0.308451,0.342781,0.405199,0.402457
HOBBIES_1_002_CA_1_validation,0.289849,0.271231,0.268241,0.269334,0.298545,0.352310,0.348956,0.288063,0.269558,0.266685,...,0.297395,0.351616,0.348762,0.288415,0.270504,0.268272,0.270174,0.300244,0.354916,0.352514
HOBBIES_1_003_CA_1_validation,0.154946,0.144994,0.143395,0.143980,0.159595,0.188337,0.186543,0.153991,0.144099,0.142564,...,0.158980,0.187966,0.186440,0.154180,0.144605,0.143412,0.144429,0.160503,0.189730,0.188446
HOBBIES_1_004_CA_1_validation,1.904414,1.782086,1.762438,1.769622,1.961548,2.314804,2.292765,1.892675,1.771095,1.752218,...,1.953995,2.310244,2.291491,1.894992,1.777313,1.762644,1.775140,1.972710,2.331926,2.316144
HOBBIES_1_005_CA_1_validation,1.091443,1.021336,1.010075,1.014192,1.124188,1.326643,1.314012,1.084716,1.015037,1.004218,...,1.119859,1.324030,1.313282,1.086043,1.018600,1.010193,1.017355,1.130585,1.336456,1.327411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3_validation,0.708080,0.662597,0.655292,0.657962,0.729323,0.860666,0.852472,0.703715,0.658510,0.651492,...,0.726514,0.858971,0.851999,0.704576,0.660822,0.655368,0.660014,0.733473,0.867033,0.861165
FOODS_3_824_WI_3_validation,0.478502,0.447767,0.442830,0.444635,0.492858,0.581617,0.576079,0.475553,0.445005,0.440262,...,0.490960,0.580471,0.575759,0.476135,0.446567,0.442881,0.446021,0.495663,0.585919,0.581954
FOODS_3_825_WI_3_validation,1.069445,1.000750,0.989717,0.993751,1.101529,1.299904,1.287528,1.062853,0.994578,0.983977,...,1.097287,1.297343,1.286812,1.064154,0.998070,0.989832,0.996849,1.107797,1.309519,1.300656


In [125]:
result_int = result_float.apply(np.round)
result_int

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_002_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_003_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOBBIES_1_004_CA_1_validation,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
HOBBIES_1_005_CA_1_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
FOODS_3_824_WI_3_validation,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
FOODS_3_825_WI_3_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [130]:
result_int = result_int.reset_index()
result_int

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,HOBBIES_1_004_CA_1_validation,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,HOBBIES_1_005_CA_1_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
30486,FOODS_3_824_WI_3_validation,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
30487,FOODS_3_825_WI_3_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
30488,FOODS_3_826_WI_3_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [137]:
for result_id, submission_id in zip(list(result_int['id']), list(sample_submission['id'])): 

    if result_id != submission_id: 
        print(result_id, submission_id)
        raise "Incorrect submmission index found"

In [141]:
result_int.to_csv)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,HOBBIES_1_004_CA_1_validation,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,HOBBIES_1_005_CA_1_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
30486,FOODS_3_824_WI_3_validation,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
30487,FOODS_3_825_WI_3_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
30488,FOODS_3_826_WI_3_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
result_int.to_csv('../data/validation_saubmission_prohphet_historical_average.csv', index=False)

In [143]:
sample_submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [144]:
pd.read_csv('../data/validation_saubmission_prohphet_historical_average.csv')

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,HOBBIES_1_004_CA_1_validation,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,HOBBIES_1_005_CA_1_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
30486,FOODS_3_824_WI_3_validation,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
30487,FOODS_3_825_WI_3_validation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
30488,FOODS_3_826_WI_3_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
